In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Desktop/PhD_Stuff/cheb_bessel_tests-main`


In [2]:
using BenchmarkTools
using LinearAlgebra
using DataInterpolations
using SpecialFunctions
using HCubature
using QuadGK
using Polynomials
using Plots
using FastChebInterp
using ProgressBars
using LaTeXStrings
using Bessels
using Tullio
using FFTW
using LoopVectorization
using NPZ
using Cubature
using FastTransforms
using Interpolations
using Dierckx
using DelimitedFiles

In [3]:
using Revise
using Will

### Adapting N5K stuff

In [4]:
z_b = npzread("background/z.npy")
χ = npzread("background/chi.npy")
z_of_χ = DataInterpolations.AkimaInterpolation(z_b, χ);

In [5]:
pk_dict = npzread("../N5K/input/pk.npz")
Pklin = pk_dict["pk_lin"]
Pknonlin = pk_dict["pk_nl"]
k = pk_dict["k"]
z = pk_dict["z"];

In [6]:
y = LinRange(log10(first(k)),log10(last(k)), length(k))
x = LinRange(first(z), last(z), length(z))
InterpPmm = Interpolations.interpolate(log10.(Pklin),BSpline(Cubic(Line(OnGrid()))))
InterpPmm = scale(InterpPmm, x, y)
InterpPmm = Interpolations.extrapolate(InterpPmm, Line());

In [7]:
y = LinRange(log10(first(k)),log10(last(k)), length(k))
x = LinRange(first(z), last(z), length(z))
InterpPmm_nl = Interpolations.interpolate(log10.(Pknonlin),BSpline(Cubic(Line(OnGrid()))))
InterpPmm_nl = scale(InterpPmm_nl, x, y)
InterpPmm_nl = Interpolations.extrapolate(InterpPmm_nl, Line());

In [8]:
power_spectrum(k, χ1, χ2) = @. sqrt(10^InterpPmm(z_of_χ(χ1),log10(k)) * 10^InterpPmm(z_of_χ(χ2),log10(k)));

In [9]:
power_spectrum_nl(k, χ1, χ2) = @. sqrt(10^InterpPmm_nl(z_of_χ(χ1),log10(k)) * 10^InterpPmm_nl(z_of_χ(χ2),log10(k)));

### Loading N5K stuff

In [10]:
#N5K Cℓ's 
benchmark_gg = npzread("../N5K/tests/benchmarks_nl_full_clgg.npz")
benchmark_ll = npzread("../N5K/tests/benchmarks_nl_full_clss.npz")
benchmark_gl = npzread("../N5K/tests/benchmarks_nl_full_clgs.npz");

In [11]:
gg = benchmark_gg["cls"]
ll = benchmark_ll["cls"]
gl = benchmark_gl["cls"]
ell = benchmark_gg["ls"];

In [12]:
gg_reshaped = zeros(length(ell), 10, 10)
counter = 1

for i in 1:10
    for j in i:10
        gg_reshaped[:,i,j] = gg[counter, :]
        gg_reshaped[:,j,i] = gg_reshaped[:,i,j]
        counter += 1
    end
end

In [13]:
ll_reshaped = zeros(length(ell), 5, 5)
counter = 1

for i in 1:5
    for j in i:5
        ll_reshaped[:,i,j] = ll[counter, :]
        ll_reshaped[:,j,i] = ll_reshaped[:,i,j]
        counter += 1
    end
end

In [14]:
gl_reshaped = zeros(length(ell), 10, 5)
counter = 1

for i in 1:10
    for j in i:5
        gl_reshaped[:,i,j] = gl[counter, :]
        gl_reshaped[:,j,i] = gl_reshaped[:,i,j]
        counter += 1
    end
end

### Functions

In [15]:
function load_Ts(folder, nχ, nR)
    ell_vector = sort!(npzread("ell_vector.npy")) 
    full_T = zeros(21, nχ, nR, 129)
    for i in 1:21
        l_string = string(round(ell_vector[i]; digits=1))
        filename = folder * "/T_tilde_l_$l_string.npy"
        full_T[i,:,:,:] = npzread(filename)
    end
    return full_T
end;

In [16]:
function SimpsonWeightArray(n)
    number_intervals = floor((n-1)/2)
    weight_array = zeros(n)
    if n == number_intervals*2+1
        for i in 1:number_intervals
            weight_array[Int((i-1)*2+1)] += 1/3
            weight_array[Int((i-1)*2+2)] += 4/3
            weight_array[Int((i-1)*2+3)] += 1/3
        end
    else
        weight_array[1] += 0.5
        weight_array[2] += 0.5
        for i in 1:number_intervals
            weight_array[Int((i-1)*2+1)+1] += 1/3
            weight_array[Int((i-1)*2+2)+1] += 4/3
            weight_array[Int((i-1)*2+3)+1] += 1/3
        end
        weight_array[length(weight_array)]   += 0.5
        weight_array[length(weight_array)-1] += 0.5
        for i in 1:number_intervals
            weight_array[Int((i-1)*2+1)] += 1/3
            weight_array[Int((i-1)*2+2)] += 4/3
            weight_array[Int((i-1)*2+3)] += 1/3
        end
        weight_array ./= 2
    end
    return weight_array
end;

In [17]:
function make_grid(χ, R)
    return vec(χ * R')
end;

In [18]:
function grid_interpolator(W, χ, grid)

    W_interp = zeros(length(W[:,1]), length(grid))
    
    for i in 1:length(W[:,1])
        interp = AkimaInterpolation(W[i,:], χ, extrapolate=true)#BSplineInterpolation(W[i,:], χ, 3, :ArcLen, :Average, extrapolate=true)
        W_interp[i,:] = interp.(grid)
    end

    return W_interp
end;

In [19]:
function grid_interpolator(W, grid, label::String)
    if label == "C"
        W_array = W["kernels_cl"]
    elseif label == "L"
        W_array = W["kernels_sh"]
    else
        error("Label must be C or L!!!!!!!")
    end

    χ = W["chi_sh"]

    return grid_interpolator(W_array, χ, grid)
end;

In [20]:
function compute_kernels(W, χ, R)

    nχ = length(χ)
    nR = length(R)
    
    W_C = reshape(grid_interpolator(W, make_grid(χ, R), "C"), 10, nχ, nR)
    
    χ2_app = zeros(5, nχ*nR)
    for i in 1:5
        χ2_app[i,:] = make_grid(χ, R) .^ 2
    end
    
    W_L = grid_interpolator(W, make_grid(χ, R), "L")
    W_L = reshape( W_L./χ2_app , 5, nχ, nR)

    

    W_C_r1 = W_C[:,:,end]
    W_L_r1 = W_L[:,:,end]

    @tullio K_CC[i,j,c,r] := W_C_r1[i,c] * W_C[j,c,r] + W_C[i,c,r]*W_C_r1[j,c]

    @tullio K_LL[i,j,c,r] := W_L_r1[i,c] * W_L[j,c,r] + W_L[i,c,r]*W_L_r1[j,c]

    @tullio K_CL[i,j,c,r] := W_C_r1[i,c] * W_L[j,c,r] + W_C[i,c,r]*W_L_r1[j,c]

    return K_CC, K_CL, K_LL
end;

In [21]:
function C_ell_computation_simpson(w, K) 
    
    nχ = length(w[1,:,1])
    nR = length(w[1,1,:])

    χ = LinRange(26,7000,nχ)
    R = LinRange(0,1, nR+1)[2:end]

    Δχ = ((last(χ)-first(χ))/(nχ-1))
    pesi_χ = SimpsonWeightArray(nχ)
    ΔR = ((last(R)-first(R))/(nR-1))
    pesi_R = SimpsonWeightArray(nR)
        
    @tullio Cℓ[l,i,j] := χ[n]*K[i,j,n,m]*w[l,n,m]*pesi_χ[n]*pesi_R[m]*Δχ*ΔR
    
    return Cℓ
    
end;

In [22]:
function C_ell_computation_simpson_uneven_grid(w, K) 
    nχ = length(w[1,:,1])
    nR = length(w[1,1,:])

    χ = LinRange(26,7000,nχ)
    R1 = LinRange(0,0.9,300)[2:end]
    R2 = LinRange(0.9,1,151)[2:end]
    R = unique(vcat( R1, R2 ))

    Δχ = ((last(χ)-first(χ))/(nχ-1))
    pesi_χ = SimpsonWeightArray(nχ)
    ΔR1 = ((last(R1)-first(R1))/(length(R1)-1))
    pesi_R1 = SimpsonWeightArray(length(R1))
    ΔR2 = ((last(R2)-first(R2))/(length(R2)-1))
    pesi_R2 = SimpsonWeightArray(length(R2))

    a = ones(length(R1))*ΔR1 
    b = ones(length(R2))*ΔR2 
    ΔR = vcat(a,b)
    pesi_R = vcat(pesi_R1,pesi_R2)
        
    @tullio Cℓ[l,i,j] := χ[n]*K[i,j,n,m]*w[l,n,m]*pesi_χ[n]*pesi_R[m]*Δχ*ΔR[m]
    
    return Cℓ
end;

In [23]:
function factorial_frac(n)
    return (n-1)*n*(n+1)*(n+2)
end;

### Loading my $w_\ell$'s

In [24]:
kmax = 200/13 
kmin = 2.5/7000
n_cheb = 128
nχ = 100
χ = LinRange(26, 7000, nχ)
Rs = unique(vcat(LinRange(0,0.9,300)[2:end], LinRange(0.9,1,151)))
nR = length(Rs)
ℓ = npzread("ell_vector.npy")[1:21]

k_cheb = chebpoints(n_cheb, log10(kmin), log10(kmax));
coeff = zeros(nχ,nR,n_cheb+1)

for i in 1:nR
    for j in 1:nχ
        c = chebinterp(power_spectrum.(10 .^ k_cheb,χ[j],χ[j]*Rs[i]), log10(kmin), log10(kmax)); #the bug was probably here, i wasn't doing 10^k_cheb
        coeff[j,i,:] = c.coefs
    end
end

In [25]:
T_LL = load_Ts("Ts_4_real/T_tilde_LL", nχ, nR+1)[:,:,2:end,:]
T_CL = load_Ts("Ts_4_real/T_tilde_CL", nχ, nR+1)[:,:,2:end,:]
T_CC = load_Ts("Ts_4_real/T_tilde_CC", nχ, nR+1)[:,:,2:end,:]


w_LL = Will.w_ell_tullio(coeff, T_LL)
w_CL = Will.w_ell_tullio(coeff, T_CL)
w_CC = Will.w_ell_tullio(coeff, T_CC);

### Limber!!

In [30]:
Cℓ_CC_limb = npzread("Cl_CC_limber_linear.npy")
Cℓ_CL_limb = npzread("Cl_CL_limber_linear.npy")
Cℓ_LL_limb = npzread("Cl_LL_limber_linear.npy");

In [32]:
Cℓ_CC_limb_nl = npzread("Cl_CC_limber_nl.npy")
Cℓ_CL_limb_nl = npzread("Cl_CL_limber_nl.npy")
Cℓ_LL_limb_nl = npzread("Cl_LL_limber_nl.npy");

# $C_\ell$'s computation

In [33]:
W = npzread("../N5K/input/kernels_fullwidth.npz")
@time K_CC, K_CL, K_LL = compute_kernels(W, χ, Rs);

  1.719164 seconds (5.62 M allocations: 441.609 MiB, 3.39% gc time, 97.44% compilation time)


In [34]:
Cℓ_CC = C_ell_computation_simpson_uneven_grid(w_CC, K_CC);
Cℓ_CL = C_ell_computation_simpson_uneven_grid(w_CL, K_CL);
Cℓ_LL = C_ell_computation_simpson_uneven_grid(w_LL, K_LL);

In [37]:
# ADDING ELL PREFACTORS
Cℓ_CC = Cℓ_CC .* (2/π)

for i in 1:length(ℓ)
    Cℓ_LL[i,:,:] = Cℓ_LL[i,:,:] .* 2 .* factorial_frac(ℓ[i]) ./ π
    Cℓ_CL[i,:,:] = Cℓ_CL[i,:,:] .* 2 .* sqrt.(factorial_frac(ℓ[i])) ./ π
end

In [38]:
final_Cℓ_CC = Cℓ_CC + Cℓ_CC_limb_nl - Cℓ_CC_limb
final_Cℓ_CL = Cℓ_CL + Cℓ_CL_limb_nl - Cℓ_CL_limb
final_Cℓ_LL = Cℓ_LL + Cℓ_LL_limb_nl - Cℓ_LL_limb;